In [69]:
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
from sklearn.datasets import make_blobs

# Data generation

In [70]:
root_synth = 'data/synthetic/'
os.makedirs(root_synth, exist_ok=True)

In [79]:
ADD_NOISE = True
N = 3000

np.random.seed(42)
for i,n_clust in tqdm(enumerate([2,3,4])):
    for j,dim in enumerate([39,70,108]): # [53,96,180]
        
        label = f'C{n_clust}_d_{dim}'

        # make data
        centers = np.pad(np.eye(n_clust), pad_width=((0,0),(0,dim - n_clust)))*2
        centers += np.random.randn(*centers.shape)*1
        X,y = make_blobs(n_samples=int(N*0.9), n_features=dim, centers=centers, cluster_std=0.5)
        if ADD_NOISE:
            X_ = X-X.mean(0)
            cov = (X_).T@X_/(X.shape[0]-1)
            cov_ = np.zeros_like(cov)
            cov_[np.diag_indices_from(cov)] = np.sqrt(cov[np.diag_indices_from(cov)])*2
            X_noise = np.random.multivariate_normal(X.mean(0), cov_, size=int(N*0.1))

            y_noise = np.zeros((int(N*0.1),))
            y_noise += max(y) + 1 # add separate class for the noise
            X = np.concatenate([X,X_noise])
            y = np.concatenate((y, y_noise))

        df = pd.DataFrame(X)
        df['target'] = y
        name = f'clust{n_clust}_dim{dim}'
        dataset_path = os.path.join(root_synth, name)
        df.to_csv(dataset_path)

3it [00:02,  1.10it/s]
